In [1]:
import requests
import json
import mysql.connector
from mysql.connector import Error
import random

In [2]:
#Pulls data from API
def getData(url):
    response = requests.get(url)
    if response.status_code == 200:
        dataJson = response.json()
        data = dataJson['data']
        return data

def chooseOwnerShip():
    random_number = random.random()
    if random_number < 0.7:
        return "renter"
    elif random_number < 0.995:
        return "owner"
    else:
        return "richAF"

#used for credit card
def convertToSQLDate(date):
    month, year = date.split('/')
    year = f"20{year}"
    sql_date = f"{year}-{month}-01"
    return sql_date
    
#after using getData() -> data, mapping is the order 
def organizePersonData(personsData):
    extracted_person_data = []
    for person in personsData:
        ownership = chooseOwnerShip()
        person_info = {
            'id': person['id'],
            'firstname': person['firstname'],
            'lastname': person['lastname'],
            'birthday': person['birthday'],
            'gender': person['gender'],
            'website': person['website'],
            'image': person['image'],
            'email': person['email'],
            'phone': person['phone'],
            'ownership': ownership # its the owner ship of the building
        }
        extracted_person_data.append(person_info)
    return extracted_person_data

def getAdditionalEmailsandPhone(personsData):
    extracted_person_data = []
    for person in personsData:
        person_info = {
            'id': person['id'],
            'email': person['email'],
            'phone': person['phone']
        }
        extracted_person_data.append(person_info)
    return extracted_person_data

def getCards(cardsData):
    extracted_card_data = []
    i = 0
    for card in cardsData:
        card_type = random.choice(["credit", "debit"])
        limit = random.randint(1,100)*1000 
        i = i + 1
        card_info = {
            'id':i,
            'type':card['type'],
            'expiration': convertToSQLDate(card['expiration']),
            'number':card['number'],
            'd_or_c':card_type, #credit or debit card
            'limit':limit, #if its credit, then its the credit limit, if its debit, its the balance
        }
        
        extracted_card_data.append(card_info)
    return extracted_card_data

def getAddresses(addressesData):
    extracted_address_data = []
    for address in addressesData:
        address_info = {
            'id':address['id'],
            'streetName':address['streetName'],
            'buildingNumber':address['buildingNumber'],
            'city':address['city'],
            'zipcode':address['zipcode'],
            'country':address['country'],
            'county_code':address['county_code'],
            'latitude':address['latitude'],
            'longitude':address['longitude']
        }
        extracted_address_data.append(address_info)
    return extracted_address_data

def getCompanies(companiesData):
    extracted_companies_data = []
    for company in companiesData:
        company_info = {
            'id':company['id'],
            'name':company['name'],
            'email':company['email'],
            'vat':company['vat'],
            'phone':company['phone'],
            'country':company['country'],
            'website':company['website'],
            'image':company['image']
        }
        extracted_companies_data.append(company_info)
    return extracted_companies_data

def getMessages(messageData):
    extracted_message_data = []
    for message in messageData:
        message_info = {
            'content':message['content']
        }
        extracted_message_data.append(message_info)
    return extracted_message_data

def connectDB():
    import mysql.connector
    from mysql.connector import Error
    # Set your database connection details HERE
    host_name = 'localhost'
    database_name = 'projectPhase1' #MYSQL database name
    user_name = 'root'  # MySQL username
    user_password = 'admin'  # MySQL password
    port = 3306
    # Establish the database connection
    try:
        connection = mysql.connector.connect(
            host=host_name,
            database=database_name,
            user=user_name,
            password=user_password,
            port=port
        )
        if connection.is_connected():
            db_Info = connection.get_server_info()
            print("Connected to MySQL Server version ", db_Info)
            cursor = connection.cursor()
            return cursor, connection
    except Error as e:
        print("Error while connecting to MySQL", e)
        return None, None

def insertInDB(select_query, query, data, cursor, connection):
    try: #try insertion
        cursor.execute(query, data)
        connection.commit()
        return cursor.lastrowid  # Return the ID of the last inserted row
    except Error as e: #duplicate, return existing one
        cursor.execute(select_query, data)
        existing_entry = cursor.fetchone()
        return existing_entry[0]

def disconnectDB(cursor, connection):
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")


In [3]:
#basic sql queries


#Person table
insert_person = ("INSERT INTO Person (firstname, lastname, birthday, gender, website, image) "
                "VALUES (%s, %s, %s, %s, %s, %s)")
select_person_id = ("SELECT id FROM Person "
                 "WHERE firstname = %s AND lastname = %s AND birthday = %s AND gender = %s AND website = %s AND image = %s")

#Email table
insert_email = ("INSERT INTO Email (personID, emailAddress) "
                "VALUES (%s, %s)")
select_email_id = ("Select id FROM Email "
                "Where personID = %s AND emailAddress = %s")

#Phone table
insert_phone_number = ("INSERT INTO PhoneNumber (personID, phoneNumber) "
                "VALUES (%s, %s)")
select_phone_number_id = ("Select id FROM PhoneNumber "
                "Where personID = %s AND phoneNumber = %s")

#Card table
insert_card = ("INSERT INTO Card (personID, number, expiration, type) "
            "VALUES (%s, %s, %s, %s)")
select_card_id = ("Select id FROM Card "
                "Where personID = %s AND number = %s AND expiration = %s AND type = %s")

#CreditCard table
insert_credit_card = ("INSERT INTO CreditCard (cardID, creditLimit, availableCredit) "
                    "VALUES (%s, %s, %s)")
select_credit_card_id = ("Select id FROM CreditCard "
                        "Where cardID = %s")

#CreditCard table
insert_debit_card = ("INSERT INTO DebitCard (cardID, balance) "
                    "VALUES (%s, %s)")
select_debit_card_id = ("Select id FROM DebitCard "
                        "Where cardID = %s")

#address table
insert_address = ("INSERT INTO Address (streetName, buildingNumber, city, zipcode, country, countyCode, latitude, longitude) "
                    "VALUES (%s, %s, %s, %s, %s, %s, %s, %s)")
select_address_id = ("Select id FROM Adress "
                    "Where streetName = %s AND buildingNumber = %s AND city = %s AND zipcode = s% AND country = %s AND countyCode = %s AND latitude = %s AND longitude = %s")

#livesAt
insert_lives_at_address = ("INSERT INTO LivesAtAddress (personID, addressID) "
                    "VALUES (%s, %s)")
select_lives_at_address_id = ("Select id FROM LivesAtAddress "
                        "Where personID = %s")

#buildingOwner
insert_building_owner = ("INSERT INTO BuildingOwner (personID, addressID) "
                        "VALUES (%s, %s)")
select_building_owner_id = ("Select id FROM BuildingOwner "
                        "Where addressID = %s")

#textMessages
insert_text_message = ("INSERT INTO TextMessages (phoneNumberID1, phoneNumberID2, messageContent) "
                        "VALUES (%s, %s, %s)")
select_text_message_id = ("Select id FROM TextMessages "
                       "Where phoneNumberID1 = %s AND phoneNumberID2 = %s AND messageContent = %s")


In [ ]:
#connections with db
cursor, connection = connectDB()

#persons data
personsData = getData("https://fakerapi.it/api/v1/persons?_quantity=400&_birthday_end=2000-01-01") # need 18yr old first
personsOrganized = organizePersonData(personsData)

#credit card data
cardsData = getData("https://fakerapi.it/api/v1/credit_cards?_quantity=400")
cardsOrganized = getCards(cardsData)

#address data
addressesData = getData("https://fakerapi.it/api/v1/addresses?_quantity=400")
addressesOrganized = getAddresses(addressesData)

#owns multiple buildings
millionaires_buildings = []

for i, person in enumerate(personsOrganized):
    #inserting in person table
    person_data = (person['firstname'], person['lastname'], person['birthday'], person['gender'], person['website'], 
                   person['image'])
    personID = insertInDB(select_person_id, insert_person, person_data, cursor, connection)
    print(personID)
    
    #associating emails to each person
    email_data = (personID, person['email'])
    emailID = insertInDB(select_email_id, insert_email, email_data, cursor, connection)
    print(emailID)
    
    #associating phone number to each person
    phone_number_data = (personID, person['phone'])
    phoneNumberID = insertInDB(select_phone_number_id, insert_phone_number, phone_number_data, cursor, connection)
    print(phoneNumberID)
    
    #associating Cards to each person
    card_info = cardsOrganized[i]
    card_data = (personID, card_info['number'], card_info['expiration'], card_info['type'])
    #print (card_data)
    cardID = insertInDB(select_card_id, insert_card, card_data, cursor, connection)
    print(cardID)
    
    #Describing whether a card is credit or debit
    #creating a card as credit or debit card
    c_or_d = card_info['d_or_c']
    limit = card_info['limit']
    if c_or_d == "credit":
        availableCredit = limit - round(random.uniform(0, limit), 2)
        credit_card_data = (cardID, limit, availableCredit)
        creditCardID = insertInDB(select_credit_card_id, insert_credit_card, credit_card_data, cursor, connection)
        print(creditCardID)
    else :
        debit_card_data = (cardID, limit)
        debitCardID = insertInDB(select_debit_card_id, insert_debit_card, debit_card_data, cursor, connection)
        print(debitCardID)
    
    #Creating addresses
    address = addressesOrganized[i]
    address_data = (address['streetName'], address['buildingNumber'], address['city'], address['zipcode'], address['country'], 
                    address['county_code'], address['latitude'],address['longitude'])
    #print(address_data)
    addressID = insertInDB(select_address_id, insert_address, address_data, cursor, connection)
    print(addressID)
    
    #Assigning where people live
    lives_at_address_data = (personID, addressID)
    lives_at_address_ID = insertInDB(select_lives_at_address_id, insert_lives_at_address, lives_at_address_data, cursor, connection)
    print(lives_at_address_ID)
    
    #Assigning owner
    person_ownership = person['ownership']
    if person_ownership == 'renter': # if he is renter, don't assign him the building. Push it into millionaires building list
        millionaires_buildings.append(addressID)
    elif person_ownership == 'owner': # owner owns the building they live in
        building_owner_data = (personID, addressID)
        building_owner_ID = insertInDB(select_building_owner_id, insert_building_owner, building_owner_data, cursor, connection)
        print(building_owner_ID)
    else: # these owner may own multiple buildings
        millionaires_buildings.append(addressID)
        while millionaires_buildings:
            addressID = millionaires_buildings.pop(0)
            building_owner_data = (personID, addressID)
            building_owner_ID = insertInDB(select_building_owner_id, insert_building_owner, building_owner_data, cursor, connection)
            print(building_owner_ID)
    
    #if last pull is not a millionaire, he won the lottery. He now is
    if i == len(personsOrganized) - 1:
        while millionaires_buildings:
            addressID = millionaires_buildings.pop(0)
            building_owner_data = (personID, addressID)
            building_owner_ID = insertInDB(select_building_owner_id, insert_building_owner, building_owner_data, cursor, connection)
            print(building_owner_ID)

disconnectDB(cursor, connection)

In [9]:
cursor, connection = connectDB()

messagesData = getData("https://fakerapi.it/api/v1/texts?_quantity=1000&_characters=5000")
messagesDataOrganized = getMessages(messagesData)

def correct(s):
    return s.replace("'", "''")

for message in messagesDataOrganized:
    text_message_info = message['content']
    text_message_cleaned = "'"+correct(text_message_info)+"'"
    person1ID = random.randint(1,3100)
    person2ID = random.randint(1,3100)
    text_message_data = (person1ID, person2ID, "'"+text_message_cleaned+"'")
    text_message_ID = insertInDB(select_text_message_id, insert_text_message, text_message_data, cursor, connection)
    print(f"PersonID1:{person1ID}, PersonID2:{person2ID}, TextMessage_id:{text_message_ID}, has been inserted")
    
disconnectDB(cursor, connection)

Connected to MySQL Server version  8.2.0
PersonID1:472, PersonID2:2763, TextMessage_id:3001, has been inserted
PersonID1:1345, PersonID2:774, TextMessage_id:3002, has been inserted
PersonID1:417, PersonID2:133, TextMessage_id:3003, has been inserted
PersonID1:1684, PersonID2:2635, TextMessage_id:3004, has been inserted
PersonID1:1365, PersonID2:300, TextMessage_id:3005, has been inserted
PersonID1:323, PersonID2:1260, TextMessage_id:3006, has been inserted
PersonID1:2406, PersonID2:1952, TextMessage_id:3007, has been inserted
PersonID1:1139, PersonID2:2495, TextMessage_id:3008, has been inserted
PersonID1:982, PersonID2:1015, TextMessage_id:3009, has been inserted
PersonID1:1083, PersonID2:1684, TextMessage_id:3010, has been inserted
PersonID1:2102, PersonID2:1262, TextMessage_id:3011, has been inserted
PersonID1:1226, PersonID2:2527, TextMessage_id:3012, has been inserted
PersonID1:1465, PersonID2:2965, TextMessage_id:3013, has been inserted
PersonID1:2865, PersonID2:328, TextMessage_

PersonID1:2859, PersonID2:1410, TextMessage_id:3146, has been inserted
PersonID1:123, PersonID2:1905, TextMessage_id:3147, has been inserted
PersonID1:911, PersonID2:2735, TextMessage_id:3148, has been inserted
PersonID1:1279, PersonID2:108, TextMessage_id:3149, has been inserted
PersonID1:1280, PersonID2:2243, TextMessage_id:3150, has been inserted
PersonID1:850, PersonID2:678, TextMessage_id:3151, has been inserted
PersonID1:2270, PersonID2:2758, TextMessage_id:3152, has been inserted
PersonID1:379, PersonID2:2357, TextMessage_id:3153, has been inserted
PersonID1:557, PersonID2:2226, TextMessage_id:3154, has been inserted
PersonID1:2438, PersonID2:1716, TextMessage_id:3155, has been inserted
PersonID1:95, PersonID2:466, TextMessage_id:3156, has been inserted
PersonID1:1181, PersonID2:25, TextMessage_id:3157, has been inserted
PersonID1:3035, PersonID2:1721, TextMessage_id:3158, has been inserted
PersonID1:1373, PersonID2:164, TextMessage_id:3159, has been inserted
PersonID1:2797, Per

PersonID1:463, PersonID2:1792, TextMessage_id:3299, has been inserted
PersonID1:2408, PersonID2:1373, TextMessage_id:3300, has been inserted
PersonID1:1601, PersonID2:2026, TextMessage_id:3301, has been inserted
PersonID1:945, PersonID2:2833, TextMessage_id:3302, has been inserted
PersonID1:2545, PersonID2:107, TextMessage_id:3303, has been inserted
PersonID1:796, PersonID2:1761, TextMessage_id:3304, has been inserted
PersonID1:2630, PersonID2:2966, TextMessage_id:3305, has been inserted
PersonID1:1245, PersonID2:2836, TextMessage_id:3306, has been inserted
PersonID1:2536, PersonID2:2367, TextMessage_id:3307, has been inserted
PersonID1:1504, PersonID2:1775, TextMessage_id:3308, has been inserted
PersonID1:238, PersonID2:1989, TextMessage_id:3309, has been inserted
PersonID1:249, PersonID2:1587, TextMessage_id:3310, has been inserted
PersonID1:517, PersonID2:2427, TextMessage_id:3311, has been inserted
PersonID1:324, PersonID2:1572, TextMessage_id:3312, has been inserted
PersonID1:2014

PersonID1:10, PersonID2:885, TextMessage_id:3440, has been inserted
PersonID1:2830, PersonID2:3025, TextMessage_id:3441, has been inserted
PersonID1:784, PersonID2:1590, TextMessage_id:3442, has been inserted
PersonID1:20, PersonID2:862, TextMessage_id:3443, has been inserted
PersonID1:2106, PersonID2:2735, TextMessage_id:3444, has been inserted
PersonID1:733, PersonID2:360, TextMessage_id:3445, has been inserted
PersonID1:309, PersonID2:657, TextMessage_id:3446, has been inserted
PersonID1:2275, PersonID2:2408, TextMessage_id:3447, has been inserted
PersonID1:2504, PersonID2:650, TextMessage_id:3448, has been inserted
PersonID1:2734, PersonID2:1057, TextMessage_id:3449, has been inserted
PersonID1:2477, PersonID2:1816, TextMessage_id:3450, has been inserted
PersonID1:1794, PersonID2:445, TextMessage_id:3451, has been inserted
PersonID1:1323, PersonID2:834, TextMessage_id:3452, has been inserted
PersonID1:2874, PersonID2:232, TextMessage_id:3453, has been inserted
PersonID1:1668, Perso

PersonID1:1546, PersonID2:1823, TextMessage_id:3580, has been inserted
PersonID1:1875, PersonID2:924, TextMessage_id:3581, has been inserted
PersonID1:1992, PersonID2:1304, TextMessage_id:3582, has been inserted
PersonID1:1679, PersonID2:1180, TextMessage_id:3583, has been inserted
PersonID1:2746, PersonID2:990, TextMessage_id:3584, has been inserted
PersonID1:230, PersonID2:1235, TextMessage_id:3585, has been inserted
PersonID1:1045, PersonID2:868, TextMessage_id:3586, has been inserted
PersonID1:2083, PersonID2:456, TextMessage_id:3587, has been inserted
PersonID1:722, PersonID2:528, TextMessage_id:3588, has been inserted
PersonID1:1592, PersonID2:483, TextMessage_id:3589, has been inserted
PersonID1:1447, PersonID2:941, TextMessage_id:3590, has been inserted
PersonID1:586, PersonID2:1671, TextMessage_id:3591, has been inserted
PersonID1:602, PersonID2:277, TextMessage_id:3592, has been inserted
PersonID1:1669, PersonID2:1717, TextMessage_id:3593, has been inserted
PersonID1:742, Per

PersonID1:3075, PersonID2:1407, TextMessage_id:3727, has been inserted
PersonID1:2526, PersonID2:2844, TextMessage_id:3728, has been inserted
PersonID1:2505, PersonID2:2387, TextMessage_id:3729, has been inserted
PersonID1:1197, PersonID2:2337, TextMessage_id:3730, has been inserted
PersonID1:1010, PersonID2:2822, TextMessage_id:3731, has been inserted
PersonID1:2765, PersonID2:1744, TextMessage_id:3732, has been inserted
PersonID1:2897, PersonID2:2962, TextMessage_id:3733, has been inserted
PersonID1:2077, PersonID2:2241, TextMessage_id:3734, has been inserted
PersonID1:161, PersonID2:1703, TextMessage_id:3735, has been inserted
PersonID1:1474, PersonID2:632, TextMessage_id:3736, has been inserted
PersonID1:637, PersonID2:2004, TextMessage_id:3737, has been inserted
PersonID1:1642, PersonID2:233, TextMessage_id:3738, has been inserted
PersonID1:2962, PersonID2:2026, TextMessage_id:3739, has been inserted
PersonID1:2945, PersonID2:830, TextMessage_id:3740, has been inserted
PersonID1:1

PersonID1:2156, PersonID2:2213, TextMessage_id:3867, has been inserted
PersonID1:2998, PersonID2:732, TextMessage_id:3868, has been inserted
PersonID1:1399, PersonID2:350, TextMessage_id:3869, has been inserted
PersonID1:2396, PersonID2:2624, TextMessage_id:3870, has been inserted
PersonID1:735, PersonID2:790, TextMessage_id:3871, has been inserted
PersonID1:1298, PersonID2:2961, TextMessage_id:3872, has been inserted
PersonID1:1121, PersonID2:2855, TextMessage_id:3873, has been inserted
PersonID1:2710, PersonID2:2249, TextMessage_id:3874, has been inserted
PersonID1:2643, PersonID2:2713, TextMessage_id:3875, has been inserted
PersonID1:2521, PersonID2:478, TextMessage_id:3876, has been inserted
PersonID1:916, PersonID2:2044, TextMessage_id:3877, has been inserted
PersonID1:2542, PersonID2:200, TextMessage_id:3878, has been inserted
PersonID1:1193, PersonID2:2202, TextMessage_id:3879, has been inserted
PersonID1:1789, PersonID2:2185, TextMessage_id:3880, has been inserted
PersonID1:171